# Generate text with zero-shot, one-shot, and few-shot inference

In [ ]:
import psutil

notebook_memory = psutil.virtual_memory()
print(notebook_memory)

if notebook_memory.total < 32 * 1000 * 1000 * 1000:
    print('*******************************************')    
    print('YOU ARE NOT USING THE CORRECT INSTANCE TYPE')
    print('PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge ')
    print('*******************************************')
else:
    correct_instance_type=True

In [179]:
%store -r setup_dependencies_passed

In [180]:
try:
    setup_dependencies_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN THE PREVIOUS NOTEBOOK ")
    print("You did not install the required libraries.   ")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [3]:
model_checkpoint = "google/flan-t5-base"
huggingface_dataset_name = "knkarthick/dialogsum"

# Load the Summarization Dataset

In [4]:
from datasets import load_dataset
dataset = load_dataset(huggingface_dataset_name)

Using custom data configuration knkarthick--dialogsum-6d41e9a7b96e340e


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-6d41e9a7b96e340e/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [32]:
example_indices = [40, 70, 80, 160,]
print('Example Input Dialogue:')
print(dataset['test'][example_indices[0]]['dialogue'])
print()
print('Example Output Summary:')
print(dataset['test'][example_indices[0]]['summary'])

Example Input Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

Example Output Summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.


# Create prompts for few-shot, one-shot, zero-shot inference on sample data

In [182]:
start_prompt = 'Summarize the following conversation.\n\n'
end_prompt = '\n\nSummary: '
stop_sequence = '---'

In [183]:
def make_prompt(num_shots):
    prompt = ''
    for i in range(num_shots + 1):
        if i == num_shots:
            dialogue = dataset['test'][example_indices[0]]['dialogue']
            summary = dataset['test'][example_indices[0]]['summary']
            prompt = prompt + f'{start_prompt}{dialogue}{end_prompt}'
        else:
            dialogue = dataset['test'][example_indices[i+1]]['dialogue']
            summary = dataset['test'][example_indices[i+1]]['summary']
            prompt = prompt + f'{start_prompt}{dialogue}{end_prompt}{summary}\n{stop_sequence}\n'
    return prompt

In [184]:
zero_shot_prompt = make_prompt(0)
print(zero_shot_prompt)

Summarize the following conversation.

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

Summary: 


In [185]:
one_shot_prompt = make_prompt(1)
print(one_shot_prompt)

Summarize the following conversation.

#Person1#: Tom, I've got good news for you.
#Person2#: What is it?
#Person1#: Haven't you heard that your novel has won The Nobel Prize?
#Person2#: Really? I can't believe it. It's like a dream come true. I never expected that I would win The Nobel Prize!
#Person1#: You did a good job. I'm extremely proud of you.
#Person2#: Thanks for the compliment.
#Person1#: You certainly deserve it. Let's celebrate!

Summary: #Person1# tells Tom that his novel has won the Nobel Prize.
---
Summarize the following conversation.

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

Summary: 


In [186]:
few_shot_prompt = make_prompt(2)
print(few_shot_prompt)

Summarize the following conversation.

#Person1#: Tom, I've got good news for you.
#Person2#: What is it?
#Person1#: Haven't you heard that your novel has won The Nobel Prize?
#Person2#: Really? I can't believe it. It's like a dream come true. I never expected that I would win The Nobel Prize!
#Person1#: You did a good job. I'm extremely proud of you.
#Person2#: Thanks for the compliment.
#Person1#: You certainly deserve it. Let's celebrate!

Summary: #Person1# tells Tom that his novel has won the Nobel Prize.
---
Summarize the following conversation.

#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napk

# Perform zero-shot, one-shot, few-shot inference BEFORE fine-tuning

In [187]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [188]:
from transformers import T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)

# Zero-shot

In [203]:
inputs = tokenizer(zero_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"], 
        max_new_tokens=50,
        do_sample=True,
        top_k=50,
        top_p=0.9
    )[0], 
    skip_special_tokens=True
)
print(f'ZERO SHOT RESPONSE: {output}')
summary = dataset['test'][example_indices[0]]['summary']
print(f'EXPECTED RESPONSE: {summary}')

ZERO SHOT RESPONSE: The train is about to stop. The train leaves at 10 o'clock and departs at 9.
EXPECTED RESPONSE: #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.


# One-shot

In [204]:
print(one_shot_prompt)

Summarize the following conversation.

#Person1#: Tom, I've got good news for you.
#Person2#: What is it?
#Person1#: Haven't you heard that your novel has won The Nobel Prize?
#Person2#: Really? I can't believe it. It's like a dream come true. I never expected that I would win The Nobel Prize!
#Person1#: You did a good job. I'm extremely proud of you.
#Person2#: Thanks for the compliment.
#Person1#: You certainly deserve it. Let's celebrate!

Summary: #Person1# tells Tom that his novel has won the Nobel Prize.
---
Summarize the following conversation.

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

Summary: 


In [220]:
inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"], 
        max_new_tokens=50,
        # eos_token_id = tokenizer(stop_sequence, return_tensors='pt'), # NOT WORKING
        do_sample=True,
        top_k=50,
        top_p=0.9
    )[0], 
    skip_special_tokens=True
)
print(f'ONE SHOT RESPONSE: {output}')
summary = dataset['test'][example_indices[0]]['summary']
print(f'EXPECTED RESPONSE: {summary}')

ONE SHOT RESPONSE: Everyone's ready to go home now.
EXPECTED RESPONSE: #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.


# Few-shot

In [216]:
inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"], 
        max_new_tokens=50,
        # eos_token_id = tokenizer(stop_sequence)['input_ids'][1], # NOT WORKING
        do_sample=True,
        top_k=50, 
        top_p=0.9
    )[0], 
    skip_special_tokens=True
)
print(f'ONE SHOT RESPONSE: {output}')
summary = dataset['test'][example_indices[0]]['summary']
print(f'EXPECTED RESPONSE: {summary}')

ONE SHOT RESPONSE: Tom doesn't know it's late to go to the railway station. He needs to catch the train, so he'll need to catch the last train at 8:00. The train will leave Tom at 10:30. ---
EXPECTED RESPONSE: #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
